In [1]:
import pgmpy
import random
def product(*args, **kwds):
    pools = [tuple(pool) for pool in args]
    result = [[]]
    for pool in pools[::-1]:
        result = [[y]+x for x in result for y in pool]
    for prod in result:
        yield tuple(prod)
from pgmpy import readwrite
from copy import deepcopy

In [3]:
net = pgmpy.readwrite.BIF.BIFReader('earthquake.bif')

In [4]:
net.variable_states

{'Burglary': ['True', 'False'],
 'Earthquake': ['True', 'False'],
 'Alarm': ['True', 'False'],
 'JohnCalls': ['True', 'False'],
 'MaryCalls': ['True', 'False']}

In [5]:
choosen_state = {}
for k, i in net.variable_states.items():
    c = random.choice(i)
    choosen_state.update({k: [c, i.index(c)]})
choosen_state

{'Burglary': ['True', 0],
 'Earthquake': ['False', 1],
 'Alarm': ['False', 1],
 'JohnCalls': ['True', 0],
 'MaryCalls': ['False', 1]}

In [6]:
def p(var, st, netw=net):
    if netw.variable_parents[var] == []:
        return netw.get_values()[var][st[1]]
    return (netw.get_values()[var][st[1]][list(product(*[netw.get_states()[idx] for idx in netw.get_parents()[var]])).index(tuple(choosen_state[i][0] for i in netw.get_parents()[var]))])


ans = 1
for i in net.get_states().keys():
    ans*=p(i, choosen_state[i])
print(ans)


[2.9106e-05]


In [11]:
miss_choosen_state = {}
for k, i in net.variable_states.items():
    if random.random()>0.5:
        c = random.choice(i)
        miss_choosen_state.update({k: [c, i.index(c)]})
    else:
        miss_choosen_state.update({k: [None, None]})
miss_choosen_state = {'Burglary': [None, None],
 'Earthquake': ['True', 0],
 'Alarm': [None, None],
 'JohnCalls': [None, None],
 'MaryCalls': ['True', 0]}

In [12]:
key_to_product = []
for k, i in miss_choosen_state.items():
    if i[1] is None: key_to_product.append(k)
st = deepcopy(miss_choosen_state)
probability = 0
for state in product(*[net.get_states()[i] for i in key_to_product]):
    for j in range(len(key_to_product)):
        st[key_to_product[j]] = [state[j], net.get_states()[key_to_product[j]].index(str(state[j]))]
    ans = 1
    for ix in net.get_states().keys():
        ans*=p(ix, st[ix])
    probability += ans
        
print(probability)

[0.0002]
